In [1]:
import numpy as np
import pandas as pd
from copy import deepcopy
from itertools import chain



In [2]:
class Graph(object):
    def __init__(self, C, C_alternative_names, C_comps, EL_comps):
        self.C = C
        self.C_alternative_names = C_alternative_names
        self.C_comps = C_comps
        self.EL_comps = EL_comps

        self.A = None

        self.init_local_weights()
        self.init_WWE()

    def init_WWE(self):
        """form the matrix of kind C_i by rows and C_j by columns"""
        eps = 1e-10
        n = len(self.C)
        self.WWE = np.block(
            [
                [
                    self.EM_local_w[(C_i, C_j)].values * self.C_weights.loc[C_i]
                    for C_j in self.C
                ]
                for C_i in self.C
            ]
        )
        self.WWE /= (self.WWE + eps).sum(axis=0)
        self.WWE = pd.DataFrame(
            self.WWE,
            columns=list(chain(*list(C_names.values()))),
            index=list(chain(*list(C_names.values()))),
        )
        return self.WWE

    def run_k_steps(self, k):
        self.weights = np.linalg.matrix_power(self.WWE, k)
        self.weights = pd.DataFrame(
            self.weights, columns=self.WWE.columns, index=self.WWE.index
        )
        return self.weights

    def get_global_weights(self, columns):
        global_weights = self.weights.loc[self.weights.index.isin(columns), columns[0]]
        global_weights /= global_weights.sum()
        return global_weights

    def init_local_weights(self):
        def EM(data):
            eps = 1e-9
            n = data.shape[0]
            index = np.argmax(np.real(np.linalg.eig(data)[0]))
            weights = np.real(np.linalg.eig(data)[1][:, index])
            weights = weights / weights.sum()
            return weights

        self.EM_local_w = {
            (C_i, C_j): pd.DataFrame(
                {
                    alternative: EM(self.EL_comps[(C_j, C_i, alternative)])
                    if (C_j, C_i, alternative) in self.EL_comps
                    and self.C_comps.loc[C_i, C_j] != 0
                    else np.zeros(len(self.C_alternative_names[C_i]))
                    for alternative in self.C_alternative_names[C_j]
                },
                index=self.C_alternative_names[C_i],
            )
            for C_i in self.C
            for C_j in self.C
        }
        self.C_weights = pd.Series(EM(self.C_comps.values), index=self.C)
        return self.EM_local_w


In [3]:
C = ["C", "A"]
C_names = {
    "C": ["price", "expenses", "lifetime"],
    "A": ["america", "european_union", "japan"],
}
C_comps = pd.DataFrame(np.array([[0, 1], [1, 0]]), index=C, columns=C)

EL_comps = {
    # Питання: який з двох типів автомобілей в більшій мірі задовольняє критерію і наскільки більше задовольняє?
    ("C", "A", "price"): np.array([[1, 5, 3], [0.2, 1, 1.0 / 3], [1.0 / 3, 3, 1]]),
    ("C", "A", "expenses"): np.array([[1, 5, 2], [0.2, 1, 1.0 / 3], [1.0 / 2, 3, 1]]),
    ("C", "A", "lifetime"): np.array(
        [[1, 1.0 / 5, 1.0 / 3], [5, 1, 3], [3, 1.0 / 3, 1]]
    ),
    # Питання: який з двох критеріїв є більш характерним для даниого типу автомобілей і наскільки більш характерним?
    ("A", "C", "america"): np.array([[1, 3, 4], [1.0 / 3, 1, 1], [1.0 / 4, 1, 1]]),
    ("A", "C", "european_union"): np.array(
        [[1, 1, 1.0 / 2], [1, 1, 1.0 / 2], [2, 2, 1]]
    ),
    ("A", "C", "japan"): np.array([[1, 2, 1], [1.0 / 2, 1, 1.0 / 2], [1, 2, 1]]),
    # Петля з випадковими вагами
    ("C", "C", "price"): np.random.randint(5, size=(3, 3)),
    ("C", "C", "expenses"): np.random.randint(5, size=(3, 3)),
    ("C", "C", "lifetime"): np.random.randint(5, size=(3, 3)),
}


In [4]:
Graph = Graph(C, C_names, C_comps, EL_comps)
Graph.run_k_steps(300)

,price,expenses,lifetime,america,european_union,japan
price,0.463711,0.463711,0.463711,0.000000,0.000000,0.000000
expenses,0.210311,0.210311,0.210311,0.000000,0.000000,0.000000
lifetime,0.325978,0.325978,0.325978,0.000000,0.000000,0.000000
america,0.000000,0.000000,0.000000,0.451823,0.451823,0.451823
european_union,0.000000,0.000000,0.000000,0.279227,0.279227,0.279227
japan,0.000000,0.000000,0.000000,0.268950,0.268950,0.268950


In [5]:
Graph.get_global_weights(["america", "european_union", "japan"])


america           0.451823
european_union    0.279227
japan             0.268950
Name: america, dtype: float64

In [6]:
Graph.get_global_weights(["price", "expenses", "lifetime"])


price       0.463711
expenses    0.210311
lifetime    0.325978
Name: price, dtype: float64

In [7]:
Graph.WWE

,price,expenses,lifetime,america,european_union,japan
price,0.000000,0.000000,0.000000,0.633708,0.25,0.4
expenses,0.000000,0.000000,0.000000,0.191921,0.25,0.2
lifetime,0.000000,0.000000,0.000000,0.174371,0.50,0.4
america,0.636986,0.581552,0.104729,0.000000,0.00,0.0
european_union,0.104729,0.109452,0.636986,0.000000,0.00,0.0
japan,0.258285,0.308996,0.258285,0.000000,0.00,0.0
